# Direct simulation of the 2-spin model on a graph

This notebook can be used to simulate the 2-spin model on a generic graph. Let $G=(V,E)$ be the graph, with $V$ being the node set and $E$ being the edge set; it is possible to associate to each node $i \in V={1,...,N}$ a degree of freedom $x_{i}(t)$ whose dynamics is described by the following equations:

\begin{equation}
    \frac{dx_{i}}{dt} = \sum_{j \in \partial i} J_{ij} x_{j}(t) - \lambda(t) x_{i}(t) + \eta_{i}(t)
\end{equation}

with $J_{ij} = J_{ji}$ being the coupling constant between node $i$ and node $j$, with $\lambda(t)$ being the Lagrange multiplier enforcing the spherical constraint ($\sum_{i}^{N} x_{i}^{2}(t) = N$ at all times) and with $\eta_{i}(t)$ being a Gaussian random variable whose properties are:


\begin{align*}
    & \langle \eta_{i}(t) \rangle = 0 \\
    & \langle \eta_{i}(t) \eta_{j}(t') \rangle = 2 D \delta_{i,j} \delta(t-t')
\end{align*}


where the parameter $D$ sets the amplitude of the fluctuations of the noise.

Here the libraries required to implement the direct numerical simulations are imported:

In [ ]:
using Random, Statistics, OffsetArrays, LinearAlgebra, Dates, DelimitedFiles, SharedArrays, Revise, Distributions, Plots, Measures, LaTeXStrings, Graphs, Interpolations, ProgressMeter, StatsBase

The the files implementing the simualations are imported:

In [ ]:
includet("simulation_2_spin.jl")
includet("simulation_data_2_spin.jl")
includet("simulation_tools_2_spin.jl")

Now the parameter of the simulation can be set:

In [ ]:
T = 10000 # number of time steps
Δ = 0.01 # time step
Nv = 1000 # number of vertices of the graph
k = 10 # connectivity of the graph

# choice of the graph§
G = random_regular_graph(Nv, k)
#G = complete_graph(Nv)

gx = x -> 0.01 # g[xᵢ(t)] term of the noise second order moment
#Jx = x -> rand() < 0.5 ? 1.0/sqrt(k) : -1.0/sqrt(k) # function that defines the couplings for the disordered case
Jx = x -> 1.0/k # function that defines the couplings for the case without disorder
x0_init = x -> 0.1 * rand() # this is the function used to initialize the variables
noise = true # true for noisy dynamics, false for deterministic dynamics

The simulation can be run as:

In [ ]:
result = simulation_2_spin(G, T, Δ, gx, Jx, x0_init, noise)

It is possible to plot the trajectories of some of the nodes of the graph as:

In [ ]:
plot_trajectories(result, 10)

It is also possible to plot the Lagrange multiplier:

In [ ]:
plot_lgm(result)

The average trajectory, the covariance matrix and the correlation matrix are easily computed:

In [ ]:
t₀ = 10
tₙ = T
μ = Vector{Float64}(undef, tₙ-t₀)
C = Matrix{Float64}(undef, (tₙ-t₀,tₙ-t₀))
ρ = Matrix{Float64}(undef, (tₙ-t₀,tₙ-t₀))

μ, C, ρ = stats_analysis(result)

The results of the analysis can be plotted:

In [ ]:
plot_stats(μ, C, ρ, t₀, tₙ)

The traejctories obtained in the simulation can be saved:

In [ ]:
folder_path = "C://your//path"
save_trajectories(result, folder_path)

The results of the statistical analysis can be saved:

In [ ]:
folder_path = "C://your//path"
save_stats(μ, C, ρ, folder_path)